In [290]:
import os
import json
from tqdm import tqdm_notebook
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from math import log

In [291]:
import warnings
warnings.filterwarnings('ignore')

In [370]:
mystem = Mystem()

In [371]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation + '»«–…') for x in word_tokenize(input_text)]
    lemmas = [mystem.lemmatize(x)[0] for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
        
    return lemmas_arr

### Inverted Index

In [297]:
avito = os.listdir('Avito')
avito = ['Avito' + os.sep + a for a in avito]

In [299]:
avito.remove('Avito/.DS_Store')

In [300]:
def prepare_data(avito):
    inv_idx_result = []
    lengths = {}
    ads_texts = []

    for a in tqdm_notebook(avito):
        with open(a, 'r', encoding='utf-8') as f:
            f = f.read()
            ads_texts.append(f)

        avito_text = preprocessing(f)
        lengths[a] = len(avito_text)
        inv_idx_result.append(' '.join(avito_text))

    return inv_idx_result, lengths, ads_texts

In [301]:
inv_idx_res, l, f = prepare_data(avito)

A Jupyter Widget

In [302]:
ads_texts = [i.replace('\xa0', '') for i in f]

In [303]:
avito_texts = []
for index, text in enumerate(ads_texts):
    avito_texts.append({'avito_text': text, 'index': index})

In [304]:
ll = dict(enumerate(l.values()))

In [305]:
avgdl = sum(l.values()) / len(l)

In [306]:
avgdl

37.3769

In [307]:
ads = dict(enumerate(avito))

In [308]:
def get_inv_index(ii_data, ads):
    count = CountVectorizer()
    X = count.fit_transform(ii_data)
    key = [k for k in ads.keys()]
    term_doc_matrix = pd.DataFrame(X.toarray(), index=key, columns=count.get_feature_names())
    term_doc_matrix = [term_doc_matrix.index.tolist(), term_doc_matrix.columns.tolist(), term_doc_matrix.values.tolist()]
    
    return term_doc_matrix

In [309]:
term_doc_matrix = get_inv_index(inv_idx_res, ads)

In [310]:
def get_inv_idx(term_doc_matrix) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    count_idf = {}
    ads = term_doc_matrix[0]
    words = term_doc_matrix[1]
    freq = term_doc_matrix[2]
    N = len(ads)

    for i, j in enumerate(words):
        n = 0
        for f in freq:
            count = f[i]
            if count != 0:
                n += 1

        idf = log((N - n + 0.5) / (n + 0.5))
        count_idf[j] = idf
    
    return count_idf

In [311]:
c_idf = get_inv_idx(term_doc_matrix)

In [312]:
from math import log

k1 = 2.0
b = 0.75

def score_BM25(idf, qf, dl, avgdl, k1, b) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    return idf * (k1 + 1) * qf / (qf + k1 * (1 - b + b * dl / avgdl))

In [357]:
def get_search_result(query, n_results) -> list:
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: list of lists with (doc_id, score)
    """
    inv_idx_result = []
    res_text = []
    
    for q in query:
        if q in term_doc_matrix[1]:
            idx = term_doc_matrix[1].index(q)
            for i, a in enumerate(term_doc_matrix[0]):
                qf = term_doc_matrix[2][i][idx]
                dl = ll[a]
                idf = c_idf[q] 
                okapi = score_BM25(idf, qf, dl, avgdl, k1, b)
                inv_idx_result.append({'index': a, 'okapi_score': okapi})
    
    res_inv_index = sorted(inv_idx_result, key=lambda k: k['okapi_score'], reverse=True)[:n_results]
    for i in res_inv_index:
        for j in avito_texts:
            if i['index'] == j['index']:
                res_text.append(j['avito_text'])
        
    
    return res_text

### Функция поиска

In [395]:
from gensim import matutils
import numpy as np 

def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

### Word2Vec

In [363]:
from gensim.models import Word2Vec

In [364]:
model_path = 'araneum_none_fasttextskipgram_300_5_2018.model'
w2v_model = Word2Vec.load(model_path)

In [365]:
def get_w2v_vectors(model, input_data):
    """Получает вектор документа"""
    vectors = []
    
    for input_d in input_data:
        try:
            vector = model.wv[input_d]
            vectors.append(vector)
        except KeyError as e:
            continue
            
    mean = sum(vectors) / len(vectors)        
    
    return mean

In [372]:
def save_w2v_base(avito):
    """Индексирует всю базу для поиска через word2vec"""
    w2v_result = []
    avito_vector = {}
    
    for a in tqdm_notebook(avito):
        with open(a, 'r', encoding='utf-8') as f:
            f = f.read()
            lemmas = preprocessing(f)
            w2v_vectors = get_w2v_vectors(w2v_model, lemmas)
            
            avito_vector = {'avito_text': f, 'w2v_vectors': w2v_vectors.tolist()}
            w2v_result.append(avito_vector)
    
    return w2v_result

In [373]:
w2v_res = save_w2v_base(avito)

A Jupyter Widget

In [374]:
def search_w2v(query, model, w2v_res, n_results):
    result = {}
    final_results = []
    get_vectors = get_w2v_vectors(w2v_model, query)
    for w2v_r in w2v_res:
        compare_similarity = similarity(get_vectors, w2v_r['w2v_vectors'])
        result[compare_similarity] = w2v_r['avito_text']
        
    for res in sorted(result, reverse=True)[:n_results]:
        final_results.append(result[res])
        
    return final_results

### Doc2Vec

In [378]:
from judicial_splitter import splitter

In [379]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [380]:
def split_d2v(avito):
    avito_text = {}
    d2v_paragraphs = []
    
    for a in tqdm_notebook(avito):
        with open(a, 'r', encoding='utf-8') as f:
            f = f.read()
            avito_text[a] = f
            splitted_text = splitter(f, 1)
            
            for text in splitted_text:
                lemmas = preprocessing(text, del_stopwords=False)
                d2v_paragraphs.append({'avito_text': f, 'avito_lemmas': lemmas})
    
    return avito_text, d2v_paragraphs

In [381]:
d2v_a, d2v_p = split_d2v(avito)

A Jupyter Widget

In [382]:
def train_doc2vec(data):
    d2v_data = [TaggedDocument(words=j['avito_lemmas'], tags=[str(i)])for i, j in enumerate(data)]

    model = Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, min_count=0, workers=4, epochs=100)
    model.build_vocab(d2v_data)
    model.train(d2v_data, total_examples=model.corpus_count, epochs=model.epochs)
    
    return model

In [383]:
d2v_model = train_doc2vec(d2v_p)

In [384]:
def get_d2v_vectors(model, input_data):
    """Получает вектор документа"""
    d2v_vectors = model.infer_vector(input_data)
    
    return d2v_vectors

In [387]:
def save_d2v_base(d2v_data):
    """Индексирует всю базу для поиска через word2vec"""
    d2v_result = []
    avito_vector = {}
    
    for dictionary in tqdm_notebook(d2v_data):
        d2v_vectors = get_d2v_vectors(d2v_model, dictionary['avito_lemmas'])
        avito_vector = {'avito_text': dictionary['avito_text'], 'd2v_vectors': d2v_vectors.tolist()}
        d2v_result.append(avito_vector)
    
    return d2v_result

In [388]:
d2v_res = save_d2v_base(d2v_p)

A Jupyter Widget

In [389]:
def search_d2v(query, model, d2v_res, n_results):
    result = {}
    final_results = []
    get_vectors = get_d2v_vectors(d2v_model, query)
    for d2v_r in d2v_res:
        compare_similarity = similarity(get_vectors, d2v_r['d2v_vectors'])
        result[compare_similarity] = d2v_r['avito_text']
        
    for res in sorted(result, reverse=True)[:n_results]:
        final_results.append(result[res])
        
    return final_results

In [399]:
def search(query, search_method, n_results=5):
    if search_method == 'inverted_index':
        query = preprocessing(query)
        result = get_search_result(query, n_results)
        print('\n\n'.join(result))
    
    elif search_method == 'word2vec':
        query = preprocessing(query)
        result = search_w2v(query, w2v_model, w2v_res, n_results)
        print('\n\n'.join(result))
    
    elif search_method == 'doc2vec':
        query = preprocessing(query, del_stopwords=False)
        result = search_d2v(query, d2v_model, d2v_res, n_results)
        print('\n\n'.join(result))
    
    else:
        raise TypeError('unsupported search method')
    
#     return final

In [400]:
search('компьютер', 'inverted_index', n_results=5)

Title: Игры для компьютера 13 шт
Date: № 931252284, размещено 1 октября в 21:10
Price: 300₽
Address: м.Выхино
Игры для компьютера 13 шт

Title: Игры на компьютер
Date: № 672540624, размещено 29 сентября в 14:05
Price: 100₽
Address: м.Коньково
Продаю 39 игровых диска для компьютора .

Title: Игры для компьютера
Date: № 1476190446, размещено 4 октября в 07:14
Price: 20₽
Address: м.Бабушкинская
Игры для компьютера. 20р. за диск

Title: Программы для компьютера
Date: № 1279195158, размещено 5 октября в 07:05
Price: 50₽
Address: м.Строгино
Программы для компьютера Цена /шт.

Title: Игры для компьютера. Разные
Date: № 197957059, размещено 2 октября в 16:21
Price: 100₽
Address: м.Бунинская аллея
Игры для компьютера.Разные. Лицензия.


In [401]:
search('компьютер', 'word2vec', n_results=5)

Title: Установочный диск Windows 7
Date: № 1212351184, размещено 2 октября в 11:52
Price: 250 ₽
Address: м. Ховрино
Загрузочный диск Windows 7 Professional x64 без вирусов, троянов, чистая установка.Официальная русская версия x64 (64 бит), установочный диск Windows 7 без вирусов, троянов и мусорных программ, но без красочных обложек и коробок. Любой способ активации, в т.ч. покупка лицензии (ключа), если нужно продам.*******************Так же делаю загрузочные флешки,- устанавливаю Windows, драйвера, программы- обучаю -  есть оригинальные программы Office, AutoCad, 3dsMAX, Photoshop, наборы программ по вашему списку на заказ и полные комплекты драйверов для ноутбуков и компьютеров- настраиваю ноутбук, компьютер- очищаю от вирусов, троянов и другой нечисти- настраиваю WIFI роутер- сборка компьютера- рем0нт компьютеров ноутбуков любой сложностиЗвоните договаривайтесь о встрече или выполнении работы по удаленке.

Title: Windows 10 pro / флешка
Date: № 1619530383, размещено 30 сентября в 0

In [402]:
search('компьютер', 'doc2vec', n_results=5)

Title: Witcher 2 Premium Edition Ведьмак 2 убийцы королей
Date: № 645523853, размещено 30 сентября в 22:15
Price: 3 000 ₽
Address: м. ВДНХ
Новое. Запечатанное. Для Компьютера.

Title: Игры для компьютера. Разные
Date: № 197957059, размещено 2 октября в 16:21
Price: 100 ₽
Address: м. Бунинская аллея
Игры для компьютера.Разные. Лицензия.

Title: Игры на компьютер
Date: № 1436509728, размещено 30 сентября в 16:03
Price: 1 ₽
Address: м. Библиотека им. Ленина
Игры на компьютер.Недорого.За ценой и вопросами в сообщения.

Title: Tom Clancy's Splinter Cell: Blacklist
Date: № 1072511172, размещено 3 октября в 07:22
Price: 1 500 ₽
Address: м. Бабушкинская
24-сантиметровая фигурка Сэма ФишераСтилбукКоллекционная коробка96-страничная графическая новелла Splinter Cell EchoesCollector's edition. Возможен обмен

Title: Sony PS2
Date: № 1520629317, размещено 7 октября в 14:22
Price: 5 500 ₽
Address: м. Профсоюзная
В хорошем состоянии, с проводами, к телевизору подключатся, с 2 джойстиками и с 5 играми